In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, set_seed

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

# Test with Phi 3.5 mini

In [2]:
MODEL_NAME = "microsoft/Phi-3.5-mini-instruct"

model = AutoModelForCausalLM.from_pretrained( 
    MODEL_NAME,  
    device_map=device,  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
template = """Below is a an example of aspect-based sentiment analysis performed on a product review of a telephone. This analysis section contains the text of the product review, followed by a JSON list containing all of the detailed analysis of the aspect targets, aspect categories, sentiment, and justification. Then there is a list of Aspect Categories: each category is a high level feature of interest to telephone company executives, such as "price" or "usability". Finally there is a query which consists of an actual telephone product review. Perform the same aspect-based sentiment analysis as shown in the example, and follow the exact same output formatting without deviation.

### EXAMPLE OF ASPECT-BASED SENTIMENT ANALYSIS
{example_absa}

### LIST OF ASPECT CATEGORIES
- price
- usability
- reliability
- environment friendliness
- aesthetic of design

### QUERY
product review: {query}
analysis output JSON list:
"""

In [7]:
example_absa = """
product review: I love how easy to use the interface of my new iPhone 4 Pro is. Unfortunately this comes with the downside that the batteries aren't great and often break down at random moments, not to mention that they are made from rare cobalt that destroys the planet! Overall a good phone for this price point.
analysis output JSON list:
[{ \"aspect target\": \"iPhone 4 Pro\", \"aspect category\": \"usability\", \"sentiment\": \"positive\"},
{ \"aspect target\": \"the batteries\", \"aspect category\": \"reliability\", \"sentiment\": \"negative\" },
{ \"aspect target\": \"the batteries\", \"aspect category\": \"environment friendliness\", \"sentiment\": \"negative\" },
{ \"aspect target\": \"iPhone 4 Pro\", \"aspect category\": \"price\", \"sentiment\": \"positive\" }]
"""

In [8]:
query = "The new Samsung galaxy is great overall, and really good value for money - I don't like how complicated the keypad is though"

# ---------


prompt = template.format(example_absa=example_absa, query=query)

messages = [ 
    {"role": "system", "content": "You are helpful asssistant that performs aspect-based sentiment analysis on telephone product reviews."}, 
    {"role": "user", "content": prompt}, 
] 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    #"do_sample": False, 
}

In [9]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [10]:
output = pipe(messages, **generation_args)

print(output[0]['generated_text'])

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


 [{ "aspect target": "Samsung galaxy", "aspect category": "usability", "sentiment": "negative" },
{ "aspect target": "Samsung galaxy", "aspect category": "price", "sentiment": "positive" }]

In this analysis, the sentiment towards the "Samsung galaxy" is assessed in two aspects:

1. Usability: The reviewer mentions that the keypad is "complicated," which indicates a negative sentiment towards the usability of the product.

2. Price: The reviewer states that the product is "really good value for money," which conveys a positive sentiment towards the price of the Samsung galaxy.

The JSON list captures these sentiments in relation to the specified aspect categories.


## Comments

Unlike phi3 mini instruct, here get the verbose explanations

So rewrite a prompt to contain it "early" then have a JSON at end

In [11]:
template2 = """Below is a an example of aspect-based sentiment analysis performed on a product review of a telephone. This analysis section contains the text of the product review, followed by a general thought process and summary of the product review that focuses on the various topics and opinions, followed by a JSON list containing all of the detailed analysis of the aspect targets, aspect categories, sentiment, and justification. Then there is a list of Aspect Categories: each category is a high level feature of interest to telephone company executives, such as "price" or "usability". Finally there is a query which consists of an actual telephone product review. Perform the same aspect-based sentiment analysis as shown in the example, and follow the exact same output formatting without deviation.

### EXAMPLE OF ASPECT-BASED SENTIMENT ANALYSIS
{example_absa}

### LIST OF ASPECT CATEGORIES
- price
- usability
- reliability
- environment friendliness
- aesthetic of design

### QUERY
product review: {query}
analysis output JSON list:
"""

example_absa2 = """
product review: I love how easy to use the interface of my new iPhone 4 Pro is. Unfortunately this comes with the downside that the batteries aren't great and often break down at random moments, not to mention that they are made from rare cobalt that destroys the planet! Overall a good phone for this price point.

analysis output:
1. general thought process: This product review contains a mixed set of opinions and opinion targets. I should think step by step. First the reviwe talks about the iPhone 4 Pro's interface, which belongs the the "usability" Aspect Category. The user mentions how easy the the interface is to use, so the sentiment towards this aspect target is positive.
2. JSON list data:
[{ \"aspect target\": \"iPhone 4 Pro\", \"aspect category\": \"usability\", \"sentiment\": \"positive\"},
{ \"aspect target\": \"the batteries\", \"aspect category\": \"reliability\", \"sentiment\": \"negative\" },
{ \"aspect target\": \"the batteries\", \"aspect category\": \"environment friendliness\", \"sentiment\": \"negative\" },
{ \"aspect target\": \"iPhone 4 Pro\", \"aspect category\": \"price\", \"sentiment\": \"positive\" }]
"""

In [13]:
prompt2 = template2.format(example_absa=example_absa2, query=query)

messages2 = [ 
    {"role": "system", "content": "You are helpful asssistant that performs aspect-based sentiment analysis on telephone product reviews."}, 
    {"role": "user", "content": prompt2}, 
] 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    #"do_sample": False, 
}

In [15]:
output2 = pipe(messages2, **generation_args)

print(output2[0]['generated_text'])

 1. general thought process: The product review expresses a generally positive opinion about the Samsung Galaxy, highlighting its value for money. However, there is a specific negative sentiment towards the "keypad" aspect of the product. The keypad falls under the "usability" aspect category.

2. JSON list data:
[{ "aspect target": "Samsung Galaxy", "aspect category": "value for money", "sentiment": "positive"},
{ "aspect target": "Samsung Galaxy", "aspect category": "usability", "sentiment": "negative" }]

In this analysis, we have identified two aspects from the product review: "value for money" and "usability". The sentiment towards "value for money" is positive, indicating that the reviewer finds the Samsung Galaxy to be a good deal. On the other hand, the sentiment towards "usability" is negative, as the reviewer expresses dissatisfaction with the complexity of the keypad. This information can be useful for telephone company executives to understand the strengths and weaknesses o